In [75]:
import pandas as pd
import matplotlib.pyplot as plt


In [62]:
# TODO: create seperate a file for preprocessed data??
df = pd.read_csv('data/average-monthly-surface-temperature.csv')
df.reset_index()

,index,Entity,Code,year,Day,Average surface temperature,Average surface temperature.1
0,0,Afghanistan,AFG,1940,1940-01-15,-2.032494,11.327695
1,1,Afghanistan,AFG,1940,1940-02-15,-0.733503,11.327695
2,2,Afghanistan,AFG,1940,1940-03-15,1.999134,11.327695
3,3,Afghanistan,AFG,1940,1940-04-15,10.199754,11.327695
4,4,Afghanistan,AFG,1940,1940-05-15,17.942135,11.327695
...,...,...,...,...,...,...,...
198895,198895,Zimbabwe,ZWE,2024,2024-08-15,20.559408,22.921250
198896,198896,Zimbabwe,ZWE,2024,2024-09-15,23.642931,22.921250
198897,198897,Zimbabwe,ZWE,2024,2024-10-15,24.407030,22.921250
198898,198898,Zimbabwe,ZWE,2024,2024-11-15,25.672321,22.921250


In [63]:
df.sample(5)

,Entity,Code,year,Day,Average surface temperature,Average surface temperature.1
144794,Poland,POL,2021,2021-03-15,3.262646,8.628550
30850,Cameroon,CMR,1960,1960-11-15,23.681591,24.086603
183725,Uganda,UGA,1950,1950-06-15,20.595362,21.156256
81469,Heard Island and McDonald Islands,HMD,2014,2014-02-15,3.264496,1.117521
193291,Venezuela,VEN,1982,1982-08-15,24.441110,24.778969


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Entity                         198900 non-null  object 
 1   Code                           198900 non-null  object 
 2   year                           198900 non-null  int64  
 3   Day                            198900 non-null  object 
 4   Average surface temperature    198900 non-null  float64
 5   Average surface temperature.1  198900 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.1+ MB


In [84]:
df.describe()

,year,Day,Average surface temperature,Average surface temperature.1
count,198900.00000,198900,198900.000000,198900.000000
mean,1982.00000,1982-06-30 14:04:14.117647104,18.072073,18.072073
min,1940.00000,1940-01-15 00:00:00,-36.240032,-21.529121
25%,1961.00000,1961-04-07 06:00:00,12.304079,10.569263
50%,1982.00000,1982-06-30 00:00:00,22.055794,21.856285
75%,2003.00000,2003-09-22 12:00:00,25.317015,25.142885
max,2024.00000,2024-12-15 00:00:00,39.889374,29.794220
std,24.53575,NaN,10.246142,8.710114


# Missing values

In [65]:
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# there are no missing values

Missing Values:
Entity                           0
Code                             0
year                             0
Day                              0
Average surface temperature      0
Average surface temperature.1    0
dtype: int64


# Duplicates

In [66]:
duplicate_rows = df[df.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

# there are no duplicate rows

Duplicate Rows:
Empty DataFrame
Columns: [Entity, Code, year, Day, Average surface temperature, Average surface temperature.1]
Index: []


# Convert Data Types

In [67]:
df["Day"] = pd.to_datetime(df["Day"])
df

,Entity,Code,year,Day,Average surface temperature,Average surface temperature.1
0,Afghanistan,AFG,1940,1940-01-15,-2.032494,11.327695
1,Afghanistan,AFG,1940,1940-02-15,-0.733503,11.327695
2,Afghanistan,AFG,1940,1940-03-15,1.999134,11.327695
3,Afghanistan,AFG,1940,1940-04-15,10.199754,11.327695
4,Afghanistan,AFG,1940,1940-05-15,17.942135,11.327695
...,...,...,...,...,...,...
198895,Zimbabwe,ZWE,2024,2024-08-15,20.559408,22.921250
198896,Zimbabwe,ZWE,2024,2024-09-15,23.642931,22.921250
198897,Zimbabwe,ZWE,2024,2024-10-15,24.407030,22.921250
198898,Zimbabwe,ZWE,2024,2024-11-15,25.672321,22.921250


# Verifying Entity and Code columns

In [ ]:
for name, num_of_occurrences in df['Entity'].value_counts().items():
    # check if every country has the same number of entries
    if num_of_occurrences != 1020:
        print(name, "has", num_of_occurrences)

for name in df["Entity"].unique():
    # check if every country name (Entity column) matches the country code 
    correct_code = df.loc[df["Entity"] == name].iloc[0]["Code"]      
    incorrect_rows = df.loc[(df["Entity"] == name) & (df["Code"] != correct_code)]
    if incorrect_rows.shape[0] > 0:
        display(incorrect_rows)


# Drop Unnecessary Columns

In [71]:
# having both "Enity" and "Code" is redundant, so I'm dropping a column
df.drop("Entity", axis=1, inplace=True, errors='ignore')
df

,Code,year,Day,Average surface temperature,Average surface temperature.1
0,AFG,1940,1940-01-15,-2.032494,11.327695
1,AFG,1940,1940-02-15,-0.733503,11.327695
2,AFG,1940,1940-03-15,1.999134,11.327695
3,AFG,1940,1940-04-15,10.199754,11.327695
4,AFG,1940,1940-05-15,17.942135,11.327695
...,...,...,...,...,...
198895,ZWE,2024,2024-08-15,20.559408,22.921250
198896,ZWE,2024,2024-09-15,23.642931,22.921250
198897,ZWE,2024,2024-10-15,24.407030,22.921250
198898,ZWE,2024,2024-11-15,25.672321,22.921250


AFG
ALB
DZA
ASM
AND
AGO
AIA
ATG
ARG
ARM
AUS
AUT
AZE
BHS
BHR
BGD
BLR
BEL
BLZ
BEN
BTN
BOL
BIH
BWA
BRA
BRN
BGR
BFA
BDI
KHM
CMR
CAN
CPV
CYM
CAF
TCD
CHL
CHN
COL
COM
COG
COK
CRI
CIV
HRV
CUB
CYP
CZE
COD
DNK
DJI
DOM
TLS
ECU
EGY
SLV
GNQ
ERI
EST
SWZ
ETH
FLK
FRO
FJI
FIN
FRA
PYF
GAB
GMB
GEO
DEU
GHA
GRC
GRL
GTM
GIN
GNB
GUY
HTI
HMD
HND
HKG
HUN
ISL
IND
IDN
IRN
IRQ
IRL
IMN
ISR
ITA
JAM
JPN
JOR
KAZ
KEN
KIR
OWID_KOS
KWT
KGZ
LAO
LVA
LBN
LSO
LBR
LBY
LTU
LUX
MDG
MWI
MYS
MLI
MRT
MUS
MEX
MDA
MNG
MNE
MAR
MOZ
MMR
NAM
NPL
NLD
NCL
NZL
NIC
NER
NGA
PRK
MKD
NOR
OMN
PAK
PSE
PAN
PNG
PRY
PER
PHL
POL
PRT
PRI
QAT
ROU
RUS
RWA
SHN
VCT
WSM
STP
SAU
SEN
SRB
SYC
SLE
SVK
SVN
SLB
SOM
ZAF
SGS
KOR
SSD
ESP
LKA
SDN
SUR
SWE
CHE
SYR
TJK
TZA
THA
TGO
TTO
TUN
TUR
TKM
UGA
UKR
ARE
GBR
USA
VIR
URY
UZB
VUT
VEN
VNM
OWID_WRL
YEM
ZMB
ZWE


# Data Plots

In [89]:
# plot average temperatures for each year
avg_temps_by_year = pd.DataFrame({"Year": [], "Avg Low Temp": [], "Avg High Temp": []})

for year in df['year'].unique():
    avg_high = df.loc[df["year"] == year]
    avg_low = df.loc[df["year"] == year]
    # bios.loc[bios["height_cm"] > 220]
    #print(year)
#df.plot(x='year', y='Average surface temperature')
avg_temps_by_year


,Year,Avg Low Temp,Avg High Temp
